In [8]:
import pandas as pd
from src.training.bert_pipeline import TrainingBertPipeline
import logging
import torch
import os

In [9]:
df = pd.read_csv("data/aes_dataset_5k_clean.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5363 entries, 0 to 5362
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   question           4859 non-null   object 
 1   reference_answer   5363 non-null   object 
 2   answer             5363 non-null   object 
 3   score              5363 non-null   float64
 4   dataset            5363 non-null   object 
 5   normalized_score   5363 non-null   float64
 6   normalized_score2  5363 non-null   int64  
 7   bert_length        5363 non-null   int64  
 8   indobert_length    5363 non-null   int64  
 9   albert_length      5363 non-null   int64  
 10  longformer_length  5363 non-null   int64  
 11  multibert_length   5363 non-null   int64  
 12  indoalbert_length  5363 non-null   int64  
dtypes: float64(2), int64(7), object(4)
memory usage: 544.8+ KB


In [10]:
df['dataset'].value_counts()

dataset
sag               2558
analisis_essay    2162
stita              333
cunlp              171
sci                139
Name: count, dtype: int64

In [3]:
# Check if the first file exists
df_result = None
if os.path.exists("experiments/results/results.csv"):
    df_result = pd.read_csv("experiments/results/results.csv")
    print(df_result['config_id'].iloc[-1])
else:
    print("File 'results.csv' does not exist.")

17
7.659122644421146e-05


In [4]:
results = []
results_epoch = []

In [5]:
batch_sizes = [4]
overlappings = [256]
epochs_list = [10]
learning_rates = [5e-5]
idx = (df_result['config_id'].iloc[-1] + 1) if df_result is not None and not df_result.empty else 0  # index untuk setiap kombinasi
ROOT_DIR = os.getcwd()

In [6]:
# model = [
#     ("bert_length", "bert-base-uncased"),
#     ("indobert_length", "indobenchmark/indobert-base-p1"),
#     ("albert_length", "albert-base-v1"),
#     ("indoalbert_length", "indobenchmark/indobert-lite-base-p2"),
#     ("longformer_length", "allenai/longformer-base-4096"),
#     ("multibert_length", "google-bert/bert-base-multilingual-uncased")
# ]

In [7]:
for batch_size in batch_sizes:
    for overlapping in overlappings:
        for num_epochs in epochs_list:
            for lr in learning_rates:
                df_result1 = None
                # Check if the second file exists
                if os.path.exists("experiments/results/results_epoch.csv"):
                    df_result1 = pd.read_csv("experiments/results/results_epoch.csv")
                    print(min(df_result1['valid_qwk']))
                else:
                    print("File 'results_epoch.csv' does not exist.")
                config = {
                    "df": df,
                    "model_name": "google-bert/bert-base-multilingual-uncased",
                    "overlapping": overlapping,
                    "batch_size": batch_size,
                    "learning_rate": lr,
                    "epochs": num_epochs,
                    "config_id": idx,
                    "max_seq_len": 512,
                    "col_length": "multibert_length",
                    "best_valid_qwk": max(df_result1['valid_qwk']) if df_result1 is not None and not df_result1.empty else float("-inf")
                }

                logging.info(
                    f"Running configuration: config_id={idx}, model_name={config['model_name']}, batch_size={batch_size}, "
                    f"max_seq_length={config['max_seq_len']}, overlapping={overlapping}, epochs={num_epochs}, learning_rate={lr}"
                )
                
                print(
                    f"\nRunning configuration: config_id={idx}, model_name={config['model_name']}, batch_size={batch_size}, "
                    f"max_seq_length={config['max_seq_len']}, overlapping={overlapping}, epochs={num_epochs}, learning_rate={lr}"
                )
                
                try:
                    pipeline = TrainingBertPipeline(config, results, results_epoch)
                    pipeline.run_training()

                    # Save results
                    # Dapatkan root project
                    results_path = os.path.join(ROOT_DIR, "experiments/results/results.csv")
                    results_epoch_path = os.path.join(ROOT_DIR, "experiments/results/results_epoch.csv")
                    TrainingBertPipeline.save_csv(results, results_path)
                    TrainingBertPipeline.save_csv(results_epoch, results_epoch_path)
                except Exception as e:
                    logging.error(f"Error in config_id={idx}: {str(e)}")
                    print(f"Error in config_id={idx}: {str(e)}")
                    torch.cuda.empty_cache()
                finally:
                    # Clear GPU memory after every configuration
                    del pipeline.model
                    del pipeline.tokenizer
                    del pipeline.optimizer
                    torch.cuda.empty_cache()

                idx += 1


Running configuration: config_id=18, model_name=bert-base-uncased, batch_size=4, max_seq_length=512, overlapping=256, epochs=10, learning_rate=5e-05
split dataset run...
create dataset run...
max len 512
max len 512
max len 512
create dataloader run...
====== Training Epoch 1/10 ======


c:\Users\User\Documents\Code\env\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Token indices sequence length is longer than the specified maximum sequence length for this model (725 > 512). Running this sequence through the model will result in indexing errors


Train Loss: 0.0482, Train QWK: 0.6507, Train Pearson: 0.7539
Validation Loss: 0.0444, Validation QWK: 0.8272, Validation Pearson: 0.8607
Model saved to experiments\models\bert-base-uncased_best_model.pt
====== Training Epoch 2/10 ======
Train Loss: 0.0276, Train QWK: 0.7420, Train Pearson: 0.8636
Validation Loss: 0.0325, Validation QWK: 0.8655, Validation Pearson: 0.8761
Model saved to experiments\models\bert-base-uncased_best_model.pt
====== Training Epoch 3/10 ======
Train Loss: 0.0187, Train QWK: 0.7835, Train Pearson: 0.9090
Validation Loss: 0.0394, Validation QWK: 0.8527, Validation Pearson: 0.8773
====== Training Epoch 4/10 ======
Train Loss: 0.0150, Train QWK: 0.8118, Train Pearson: 0.9276
Validation Loss: 0.0344, Validation QWK: 0.8513, Validation Pearson: 0.8892
====== Training Epoch 5/10 ======
Train Loss: 0.0120, Train QWK: 0.8253, Train Pearson: 0.9426
Validation Loss: 0.0410, Validation QWK: 0.8295, Validation Pearson: 0.8947
====== Training Epoch 6/10 ======
Train Loss: 0